In [2]:
df = spark.sql("SELECT * FROM bing_news_db.tbl_latest_news LIMIT 1000")
display(df)

StatementMeta(, b0bc3b4b-25f9-4334-a36a-57cafebeaac9, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 418d04bb-eb43-47ed-b73a-01af4ffae27b)

In [3]:
display(df.select("snippet"))

StatementMeta(, b0bc3b4b-25f9-4334-a36a-57cafebeaac9, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b0f0af14-0474-4b4d-b946-b5b706ff917f)

In [4]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, b0bc3b4b-25f9-4334-a36a-57cafebeaac9, 6, Finished, Available, Finished)

In [5]:
model = (AnalyzeText()
        .setTextCol("snippet")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("Error"))

StatementMeta(, b0bc3b4b-25f9-4334-a36a-57cafebeaac9, 7, Finished, Available, Finished)

In [6]:
result = model.transform(df)

StatementMeta(, b0bc3b4b-25f9-4334-a36a-57cafebeaac9, 8, Finished, Available, Finished)

In [7]:
display(result)

StatementMeta(, b0bc3b4b-25f9-4334-a36a-57cafebeaac9, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0c6ce336-b2e9-49d2-a80e-f7a6031777be)

In [8]:
display(result.select("snippet","response"))

StatementMeta(, b0bc3b4b-25f9-4334-a36a-57cafebeaac9, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f7a7fb48-2751-4653-b5fe-07738a48614c)

In [9]:
from pyspark.sql.functions import col
sentiment_df = result.withColumn("Sentiment",col("response.documents.sentiment"))

StatementMeta(, b0bc3b4b-25f9-4334-a36a-57cafebeaac9, 11, Finished, Available, Finished)

In [10]:
display(sentiment_df)

StatementMeta(, b0bc3b4b-25f9-4334-a36a-57cafebeaac9, 12, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2c0edfcb-37e7-4b79-9864-d72c2c396e21)

In [11]:
df_final = sentiment_df.drop("error","response")

StatementMeta(, b0bc3b4b-25f9-4334-a36a-57cafebeaac9, 13, Finished, Available, Finished)

In [12]:
display(df_final)

StatementMeta(, b0bc3b4b-25f9-4334-a36a-57cafebeaac9, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 88c4c9eb-e0fa-4dda-b069-c4e34a847226)

In [13]:
display(df_final.select("snippet","Sentiment"))

StatementMeta(, b0bc3b4b-25f9-4334-a36a-57cafebeaac9, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5fcc4c39-35f7-408d-a276-edf0894f6ca6)

In [14]:
from pyspark.sql.utils import AnalysisException

table_name = "bing_news_db.tbl_sentiment_analysis"

spark.sql("CREATE DATABASE IF NOT EXISTS bing_news_db")

try:
    # Try creating the table fresh
    df_final.write.format("delta").saveAsTable(table_name)
    print(f"Table {table_name} created successfully.")

except AnalysisException:
    print(f"Table {table_name} already exists. Dropping and recreating...")

    # Drop the existing table
    spark.sql(f"DROP TABLE IF EXISTS {table_name}")

    # Recreate with df_final
    df_final.write.format("delta").saveAsTable(table_name)
    print(f"Table {table_name} recreated successfully.")

StatementMeta(, b0bc3b4b-25f9-4334-a36a-57cafebeaac9, 16, Finished, Available, Finished)

Table bing_news_db.tbl_sentiment_analysis already exists. Dropping and recreating...
Table bing_news_db.tbl_sentiment_analysis recreated successfully.


In [15]:
%%sql
SELECT * FROM tbl_sentiment_analysis

StatementMeta(, b0bc3b4b-25f9-4334-a36a-57cafebeaac9, 17, Finished, Available, Finished)

<Spark SQL result set with 51 rows and 14 fields>